In [1]:
# analyze specific examples (e.g. 5) for our different kg setups
# look at all and subset that likely to need kg (avg rouge)
# output subset scores at different thresholds
import json

# experiment_dir = "/dccstor/srosent2/primeqa-mengxia/experiments/oracle-kg/analyze/"
#experiments = ["/default/","all/kg_vocab/", "all/kg_triples", "all/kg_sentences", "all/kg_sentences-train_more_eps3/dev"]
# experiments = ["all/kg_sentences-train_more_eps3/5ktrain/"]

# experiment_dir = "/dccstor/srosent2/primeqa-mengxia/experiments/filter_pr/"
# experiments = ["/dccstor/srosent2/primeqa-mengxia/experiments/oracle-kg/analyze/default/", experiment_dir + "all/kg_vocab_score_cased", experiment_dir + "all/kg_triple_score_cased", 
# experiment_dir + "best/kg_vocab_score_cased", experiment_dir + "best/kg_triple_score_cased"]


experiment_dir = "/dccstor/srosent2/primeqa-mengxia/experiments/filter_pr/"
experiments = ["/dccstor/srosent2/primeqa-mengxia/experiments/asqa/default-cased/"]

prediction_file = "/output/eval_predictions.json"
reference_file = "/output/eval_references.json"

threshold_file = "/dccstor/srosent2/primeqa-mengxia/experiments/oracle-kg/analyze/dev_id2score.json"
threshold_file = "/dccstor/srosent2/primeqa-mengxia/data/asqa/passages_kg_pr/dev_id2score.json"

dev_file = "/dccstor/srosent2/primeqa-mengxia/data/dpr-100passages_withkg_best_all/eli5-dev-kilt-dpr-kg-00.json"
# dev_file = "/dccstor/srosent2/primeqa-mengxia/data/dpr-100passages_withkg_best_all/eli5-train-kilt-dpr-kg-00.json"

def load_json(file_name):
    print(file_name)
    with open (file_name, 'r') as f:
        data_lines = json.load(f)
            
    return data_lines

def load_jsonl(file_name):
    print(file_name)
    json_lines = []
    with open (file_name, 'r') as f:
        data_lines = f.readlines()
        for line in data_lines:
           json_lines.append(json.loads(line))
    return json_lines

In [2]:
all_prediction_data = {}

for experiment in experiments:
    all_prediction_data[experiment] = load_json(experiment + prediction_file)

with open (threshold_file, 'r') as f:
    threshold_data = json.load(f)
reference_data = load_json(experiments[0] + reference_file)

dev_data = load_jsonl(dev_file)

/dccstor/srosent2/primeqa-mengxia/experiments/asqa/default-cased//output/eval_predictions.json
/dccstor/srosent2/primeqa-mengxia/experiments/asqa/default-cased//output/eval_references.json
/dccstor/srosent2/primeqa-mengxia/data/dpr-100passages_withkg_best_all/eli5-dev-kilt-dpr-kg-00.json


In [5]:

def compute_scores(data, thresholds, threshold=0):
    rouge_scores = {}
    for experiment in data:
        rougeL = 0
        num_instances = 0
        for instance in data[experiment]:
            if thresholds[instance['id']][1] >= threshold:
                rougeL += instance['rougeL']
                num_instances += 1
        if num_instances == 0:
            rouge_scores[experiment] = 0
        else:
            rouge_scores[experiment] = rougeL / num_instances
    return rouge_scores, num_instances

In [6]:

for i in range(0,10):
   
    rouge_scores, num_instances = compute_scores(all_prediction_data, threshold_data, i*.1)

    scores = ""
    if i == 0:
        scores += "threshold, num instances, "
        for score in rouge_scores:
            scores += str(score[score.rindex("/"):]) + ","
        scores += "\n"    
    scores += str(i*.1) + "," + str(num_instances) + ","
    for score in rouge_scores:
        scores += str(rouge_scores[score]) + ","
    print(scores)


threshold, num instances, /,
0.0,948,0.4043821532860462,
0.1,511,0.3576703141808277,
0.2,195,0.32954506918081733,
0.30000000000000004,36,0.3188683163907594,
0.4,5,0.23831348530076615,
0.5,2,0.23072186709861459,


ZeroDivisionError: division by zero

In [4]:
def get_best_answer(answers, filter="rouge"):
    best_answer = None
    best_score = 0

    for answer in answers:
        if filter == "rouge":
            if answer['meta']['rouge'] > best_score:
                best_score = answer['meta']['rouge']
                best_answer = answer
        elif filter == "size_kg":
            if len(answer['kg_vocab']) > best_score:
                best_score = len(answer['kg_vocab'])
                best_answer = answer
    return best_answer

def get_best_kg_data(instances, text_column='sentence', threshold=2):
    output = ""
    for instance in instances:
        if instance['count'] >= threshold:
            output += instance[text_column] + " (" + str(instance['count']) + ")\n"
    return output

In [3]:
import pandas as pd

# keep 5-10 examples to analyze the output from each system.
count = 0
index = 0
all_output = []
for instance in reference_data:
    output = []
    # if threshold_data[instance['id']] < 0.4:
    #     index+=1
    #     continue
    # if count < 15:
    output.append(instance['id'])
    output.append(dev_data[index]['input'])
    #output.append(threshold_data[instance['id']])
    answers = ""

    for answer in instance['answers']:
        answers += "ANSWER: " + answer + "\n"
    output.append(answers)

    answer = get_best_answer(dev_data[index]["output"], filter='size_kg')

    output.append(answer['answer'])
    # output.append(answer['meta']['rouge']/len(answers))
    output.append(answer['kg_vocab'])
    output.append(get_best_kg_data(answer['kg_triples'], text_column='text'))
    output.append(get_best_kg_data(answer['kg_sentences']))

    for experiment in all_prediction_data:
        
        pred_instance = all_prediction_data[experiment][index]

        output.append(pred_instance['prediction_text'])
        output.append(pred_instance['rougeL'])
        count += 1
    all_output.append(output)
    # if count >= 15:
    #     break
    index+=1
with pd.ExcelWriter(experiment_dir + "/analysis-dev-filtering.xlsx", engine='xlsxwriter', options={'strings_to_formulas':False}) as writer:
        df = pd.DataFrame(all_output)
        df.to_excel(writer, sheet_name='Sheet1') #columns=["id","question","answers","default","RougeL","kg vocab","RougeL","kg triples","Rouge L","kg sentences","Rouge L"])
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        cell_format = workbook.add_format({'text_wrap': True})
        worksheet.set_column('A:Z', cell_format=cell_format)
print(len(all_output))

KeyError: '1oy5tc'